In [1]:
%cd ..

/Users/Matteo/PycharmProjects/TrafficEmu


# Bayesian optimization: finding the minimum emission

In this notebook we will performing bayesian optimization on the emulator trained on our sumo simulation. We are interested in identifying the parameters which enable us to obtain the minimum travel time. Our emulator is a gaussian process with an **rbf** kernel.


## Imports

In [2]:
import pickle
import numpy as np
import emukit as ek
import GPy
import pandas as pd

from emukit.model_wrappers import GPyModelWrapper
from emukit.core.initial_designs import RandomDesign
from emukit.core import ParameterSpace, ContinuousParameter, DiscreteParameter
from emukit.bayesian_optimization.acquisitions import ExpectedImprovement
from emukit.bayesian_optimization.loops import BayesianOptimizationLoop

from experimental_design import config
from sumo_grid_simulation.grid_simulation import Simulator

warning in stationary: failed to import cython module: falling back to numpy
warning in coregionalize: failed to import cython module: falling back to numpy
warning in choleskies: failed to import cython module: falling back to numpy


## Simulator

We start by initialising our simulation.

In [3]:
simulator = Simulator(end_time=300)

We then define the two user functions in which we are interested:

In [4]:
def user_function_emissions(X):
    """  X = inputs - emukit doesnt pass named args, just an NxM ndarray, N is the number of points to evaluate, M is the number of parameters per each point """
    result = []
    i = 0
    
    print(X)
    print(f'\nUser function emissions called with {X.shape[0]} inputs to simulate')

    
    for gridSize, edgeMaxSpeed, maxSpeed, edgeLength, numLanes, accel in X:
        print(f'\nEvaluating input: {i+1} of {X.shape[0]}\n')
        
        alpha = 0.005
        max_number_of_vehicles = ((gridSize - 1) * gridSize * 2 + 4 * gridSize) * edgeLength / 5
        period = 300/(max_number_of_vehicles * alpha)
        
        s = simulator.simulate(
            gridSize      = int(gridSize),
            edgeMaxSpeed  = edgeMaxSpeed,
            maxSpeed      = maxSpeed,
            edgeLength    = edgeLength,
            numberOfLanes = int(numLanes),
            accel         = accel,
            trips_generator_period = period
        )

        CO2 = s['CO2']/s['num_emissions_samples']
        result.append(CO2/(s['duration'] * s['speed']))
        print(f'\nOutput {result[-1]}\n')
        i += 1
        
    # expand dims is essential or the acquition function breaks
    return np.expand_dims(np.array(result), 1)  

We then load the parameters' space from our configuration file:

In [5]:
parameter_space = config.get_parameter_space()

## Emulator

### Rnadomly initialised emulator

The first emulator we are going to analyse is the one that is just initilised with random points

We first sample at random 200 datapoints from the parameter space:

In [6]:
init_X, init_Y = pickle.load(open('bayesian_optimization_new_outputs/init_points/270_init_points_emissions.pkl', 'rb'))

'''
design = RandomDesign(parameter_space)
num_init_points = 270
init_X = design.get_samples(num_init_points)
init_Y = user_function_emissions(init_X)
init_points = init_X, init_Y
with open(f'bayesian_optimization_new_outputs/init_points/{num_init_points}_init_points_emissions.pkl', "wb") as f:
     pickle.dump(init_points, f)
'''

[[ 5.         12.18082649 18.8468186  46.99582856  1.          4.50929979]
 [11.         10.76974349  5.24535327 69.22592999  1.          2.76330149]
 [ 5.         21.61067248 17.41779625 40.10987083  2.          4.80918131]
 ...
 [18.         17.8298525  21.66135356 59.21727758  2.          3.68492066]
 [19.         16.92469248 16.06943223 51.62553142  2.          2.15138894]
 [14.         11.82983431  6.21990632 62.10346949  1.          3.93177032]]

User function emissions called with 270 inputs to simulate

Evaluating input: 1 of 270

 Retrying in 1 seconds

Output 16.198958033709683


Evaluating input: 2 of 270

 Retrying in 1 seconds

Output 2.0424693061100547


Evaluating input: 3 of 270

 Retrying in 1 seconds

Output 23.93412234128901


Evaluating input: 4 of 270

 Retrying in 1 seconds

Output 2.206168343473036


Evaluating input: 5 of 270

 Retrying in 1 seconds

Output 13.84551312197348


Evaluating input: 6 of 270

 Retrying in 1 seconds

Output 9.784658857278439


Evaluat

 Retrying in 1 seconds

Output 7.272966175545103


Evaluating input: 98 of 270

 Retrying in 1 seconds

Output 4.334927252302753


Evaluating input: 99 of 270

 Retrying in 1 seconds

Output 6.430221674998059


Evaluating input: 100 of 270

 Retrying in 1 seconds

Output 10.378611912085091


Evaluating input: 101 of 270

 Retrying in 1 seconds

Output 5.150417091133974


Evaluating input: 102 of 270

 Retrying in 1 seconds

Output 10.339029236478941


Evaluating input: 103 of 270

 Retrying in 1 seconds

Output 10.291877639413197


Evaluating input: 104 of 270

 Retrying in 1 seconds

Output 7.7446421161589205


Evaluating input: 105 of 270

 Retrying in 1 seconds

Output 5.067074237193671


Evaluating input: 106 of 270

 Retrying in 1 seconds

Output 8.817525320353884


Evaluating input: 107 of 270

 Retrying in 1 seconds

Output 18.811929348045247


Evaluating input: 108 of 270

 Retrying in 1 seconds

Output 9.325407298072054


Evaluating input: 109 of 270

 Retrying in 1 seconds

O

 Retrying in 1 seconds

Output 2.5827606002725156


Evaluating input: 199 of 270

 Retrying in 1 seconds

Output 6.325421405717069


Evaluating input: 200 of 270

 Retrying in 1 seconds

Output 13.17459526237965


Evaluating input: 201 of 270

 Retrying in 1 seconds

Output 6.86865890342591


Evaluating input: 202 of 270

 Retrying in 1 seconds

Output 3.625164014766223


Evaluating input: 203 of 270

 Retrying in 1 seconds

Output 4.443386617750007


Evaluating input: 204 of 270

 Retrying in 1 seconds

Output 2.746036244112864


Evaluating input: 205 of 270

 Retrying in 1 seconds

Output 12.608793103066356


Evaluating input: 206 of 270

 Retrying in 1 seconds

Output 4.530953235484135


Evaluating input: 207 of 270

 Retrying in 1 seconds

Output 7.646020456246202


Evaluating input: 208 of 270

 Retrying in 1 seconds

Output 3.141467333904541


Evaluating input: 209 of 270

 Retrying in 1 seconds

Output 11.926877952889852


Evaluating input: 210 of 270

 Retrying in 1 seconds

Ou

To then fit a GP to these points

In [7]:
emulator = GPy.models.GPRegression(init_X, init_Y, noise_var=1e-10)
emukit_model_random_init = GPyModelWrapper(emulator, n_restarts=5)
emukit_model_random_init.optimize()

Optimization restart 1/5, f = 658.0299306274745
Optimization restart 2/5, f = 508.81290764577227
Optimization restart 3/5, f = 508.81290764579353
Optimization restart 4/5, f = 508.81290764583923
Optimization restart 5/5, f = 984.7064100908477


### Experimentally designed emulator with model variance

We also load the emulator obtained during experimental design:

In [8]:
emukit_model_variance = pickle.load(open('experimental_design_new_outputs/models/model_variance_20_init_points_250_loops_co2.pkl', 'rb'))

### Experimentally designed emulator with integrated variance reduction

We also load the emulator obtained during experimental design:

In [9]:
emukit_model_integrated_variance = pickle.load(open('experimental_design_new_outputs/models/integrated_variance_reduction_20_init_points_250_loops_co2.pkl', 'rb'))

## Bayesian optimisation

We now run bayesian optimisation on the emulators

In [10]:
n_iter_bo = 50

### Acquisition functions

In [11]:
acquisition_random_init = ExpectedImprovement(emukit_model_random_init)

In [12]:
acquisition_model_variance = ExpectedImprovement(emukit_model_variance)

In [13]:
acquisition_integrated_variance = ExpectedImprovement(emukit_model_integrated_variance)

### Optimisation loop

In [14]:
bo_random_init = BayesianOptimizationLoop(parameter_space, emukit_model_random_init, acquisition=acquisition_random_init)
bo_random_init.run_loop(user_function_emissions, n_iter_bo)

Optimization restart 1/5, f = 508.81290764577227
Optimization restart 2/5, f = 508.8129082007515
Optimization restart 3/5, f = 508.8129076464255
Optimization restart 4/5, f = 508.8129076515597
Optimization restart 5/5, f = 508.81290766214806
[[20. 25.  5. 70.  1.  5.]]

User function emissions called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 1.4793876601068827

Optimization restart 1/5, f = 510.98189268139697
Optimization restart 2/5, f = 510.97817157159284
Optimization restart 3/5, f = 510.9781715719239
Optimization restart 4/5, f = 510.9781715717382
Optimization restart 5/5, f = 510.9781717116666
[[12.         25.          5.         66.83379469  1.          1.5       ]]

User function emissions called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 1.9186259626999325

Optimization restart 1/5, f = 513.3521339239195
Optimization restart 2/5, f = 513.3487243425573
Optimization restart 3/5, f = 513.3487243419

 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square


Optimization restart 4/5, f = 538.0748824978165
Optimization restart 5/5, f = 538.0748824488542
[[20.   8.   5.  70.   1.   1.5]]

User function emissions called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 1.19946831512822

Optimization restart 1/5, f = 539.4703927423878
Optimization restart 2/5, f = 539.4702279004639
Optimization restart 3/5, f = 682.5255291164048
Optimization restart 4/5, f = 539.4702279218241
Optimization restart 5/5, f = 539.4702278949692
[[13.  25.   5.  70.   1.   1.5]]

User function emissions called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 1.6353728063663815

Optimization restart 1/5, f = 540.6395720505662
Optimization restart 2/5, f = 540.6395569437016
Optimization restart 3/5, f = 540.6395569423507
Optimization restart 4/5, f = 540.6395569470019
Optimization restart 5/5, f = 540.6395569458522
[[20.          8.         11.49241483 70.          1.          1.5       ]]

User func

 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square


Optimization restart 2/5, f = 554.3595208716254
Optimization restart 3/5, f = 554.3595208710447
Optimization restart 4/5, f = 554.3595208710396
Optimization restart 5/5, f = 554.3595208717658
[[14.          8.         27.         66.12674287  3.          1.5       ]]

User function emissions called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 2.4303479202821583

Optimization restart 1/5, f = 555.8848620342366
Optimization restart 2/5, f = 555.883457034334
Optimization restart 3/5, f = 555.8834570403448
Optimization restart 4/5, f = 555.8834570347575
Optimization restart 5/5, f = 555.8834570343756
[[20.   8.   5.  70.   1.   1.5]]

User function emissions called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 1.19946831512822

Optimization restart 1/5, f = 556.9406257345227
Optimization restart 2/5, f = 556.940610618756
Optimization restart 3/5, f = 556.9406106660267
Optimization restart 4/5, f = 556.940610638586

 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square


Optimization restart 4/5, f = 559.0451220501598
Optimization restart 5/5, f = 559.0451220500036
[[20.         15.69660837  5.         70.          3.          1.5       ]]

User function emissions called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 1.3097480384484506

Optimization restart 1/5, f = 560.0613715080707
Optimization restart 2/5, f = 560.061360667503
Optimization restart 3/5, f = 635.1311454751499
Optimization restart 4/5, f = 560.0613606843735
Optimization restart 5/5, f = 560.0613606679854
[[20.         13.76139069  5.         70.          1.          1.5       ]]

User function emissions called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 1.19946831512822

Optimization restart 1/5, f = 561.0650449694704
Optimization restart 2/5, f = 561.0650345435467
Optimization restart 3/5, f = 561.0650345106724
Optimization restart 4/5, f = 561.065034510397
Optimization restart 5/5, f = 561.0650345404708
[[20

In [15]:
bo_model_variance = BayesianOptimizationLoop(parameter_space, emukit_model_variance, acquisition=acquisition_model_variance)
bo_model_variance.run_loop(user_function_emissions, n_iter_bo)

Optimization restart 1/5, f = 704.2580691975602


 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


Optimization restart 2/5, f = 704.2580692149708
Optimization restart 3/5, f = 1020.1398153113507
Optimization restart 4/5, f = 1020.139825969681
Optimization restart 5/5, f = 704.2580692008632
[[20.   8.   5.  70.   1.   1.5]]

User function emissions called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 1.19946831512822

Optimization restart 1/5, f = 706.0455544000944


 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


Optimization restart 2/5, f = 697.7844048993775
Optimization restart 3/5, f = 1012.1949910112664
Optimization restart 4/5, f = 706.0454584986635
Optimization restart 5/5, f = 706.0454585054285
[[15.        25.         5.        63.9449529  1.         1.5      ]]

User function emissions called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 1.5373330905228264

Optimization restart 1/5, f = 699.4369615641347
Optimization restart 2/5, f = 707.7901409352478
Optimization restart 3/5, f = 707.7901409343731
Optimization restart 4/5, f = 707.7901409409359
Optimization restart 5/5, f = 707.7901409345912
[[14.          8.          5.         48.58966385  3.          5.        ]]

User function emissions called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 3.002053057130501

Optimization restart 1/5, f = 702.553155640471
Optimization restart 2/5, f = 709.5335225296917
Optimization restart 3/5, f = 709.5335225297098
Optimiz

...

Optimization restart 2/5, f = 753.1561933270957
Optimization restart 3/5, f = 753.156193327123
Optimization restart 4/5, f = 1109.832669170393
Optimization restart 5/5, f = 753.156193327932
[[19.         17.41576652  5.1665547  65.10916317  1.          2.5489973 ]]

User function emissions called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 1.5856210337219705

Optimization restart 1/5, f = 737.9734879606973
Optimization restart 2/5, f = 737.9726810868046
Optimization restart 3/5, f = 754.7135401407279
Optimization restart 4/5, f = 737.972681799411
Optimization restart 5/5, f = 754.7135401444027
[[20.          8.         17.12772901 70.          1.          1.5       ]]

User function emissions called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 1.5618519313986508

Optimization restart 1/5, f = 738.9466294593451
Optimization restart 2/5, f = 756.3397421210467
Optimization restart 3/5, f = 756.3397423135852
Opt

 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in true_divide
 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


Optimization restart 5/5, f = 848.8206482677639
[[15.         11.03323153 27.         70.          1.          1.5       ]]

User function emissions called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 2.535059092284798

Optimization restart 1/5, f = 757.2968165423795
Optimization restart 2/5, f = 776.0476104246975
Optimization restart 3/5, f = 776.0476104245258
Optimization restart 4/5, f = 776.0476104271127
Optimization restart 5/5, f = 776.0476104255022
[[20.         25.          8.93144389 61.01906608  1.          1.5       ]]

User function emissions called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 2.202446496642129

Optimization restart 1/5, f = 758.4580934432956
Optimization restart 2/5, f = 1161.7611303808035
Optimization restart 3/5, f = 777.6024052092224
Optimization restart 4/5, f = 777.6024052172219
Optimization restart 5/5, f = 777.6024052239918
[[15.         13.51291287  5.         70.        

 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square


Optimization restart 3/5, f = 782.1937652511127
Optimization restart 4/5, f = 782.1937652708826
Optimization restart 5/5, f = 782.193765250667
[[11.          8.         23.53602105 36.64320813  3.          1.5       ]]

User function emissions called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 6.024265073218446

Optimization restart 1/5, f = 764.2590115444166
Optimization restart 2/5, f = 784.2853530878951
Optimization restart 3/5, f = 784.2853530826945
Optimization restart 4/5, f = 764.2590101658581
Optimization restart 5/5, f = 784.2853530724637
[[20.          8.         21.00485706 66.64699075  1.          5.        ]]

User function emissions called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 1.9684076752403847

Optimization restart 1/5, f = 765.1944968733742
Optimization restart 2/5, f = 785.8376993248412
Optimization restart 3/5, f = 785.8376993259001
Optimization restart 4/5, f = 785.8376993225227
Op

In [16]:
bo_integrated_variance = BayesianOptimizationLoop(parameter_space, emukit_model_integrated_variance, acquisition=acquisition_integrated_variance)
bo_integrated_variance.run_loop(user_function_emissions, n_iter_bo)

Optimization restart 1/5, f = 667.0617016290553
Optimization restart 2/5, f = 667.0617016446129
Optimization restart 3/5, f = 667.0617016292816
Optimization restart 4/5, f = 667.0617016290481
Optimization restart 5/5, f = 667.0617016307854
[[20.  25.   5.  70.   1.   1.5]]

User function emissions called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 1.19946831512822

Optimization restart 1/5, f = 669.3486546094341
Optimization restart 2/5, f = 669.3486339656895
Optimization restart 3/5, f = 669.3486339657986
Optimization restart 4/5, f = 669.3486339660918
Optimization restart 5/5, f = 669.3486339655851
[[20.         18.28660697  5.         70.          1.          1.5       ]]

User function emissions called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 1.19946831512822

Optimization restart 1/5, f = 671.1591233473321
Optimization restart 2/5, f = 671.1590924742927
Optimization restart 3/5, f = 671.159092475862

 Retrying in 1 seconds

Output 1.19946831512822

Optimization restart 1/5, f = 701.2858737683455
Optimization restart 2/5, f = 701.285868296491
Optimization restart 3/5, f = 701.28586830491
Optimization restart 4/5, f = 701.2858682964068
Optimization restart 5/5, f = 701.2858683000754
[[20.         21.03565926  5.         70.          1.          1.5       ]]

User function emissions called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 1.19946831512822

Optimization restart 1/5, f = 702.828215482946
Optimization restart 2/5, f = 702.828208910629
Optimization restart 3/5, f = 702.8282088648889
Optimization restart 4/5, f = 702.8282088651506
Optimization restart 5/5, f = 702.8282088647895
[[20.  25.   5.  70.   3.   1.5]]

User function emissions called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 1.3111349350393235

Optimization restart 1/5, f = 704.5375140635272
Optimization restart 2/5, f = 704.5375018433879


Optimization restart 5/5, f = 704.780090979414
[[20.         16.10404883  5.         58.43422711  1.          1.5       ]]

User function emissions called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 1.4847144211996919

Optimization restart 1/5, f = 707.2178034231815
Optimization restart 2/5, f = 732.4075668331931
Optimization restart 3/5, f = 732.4075668326373
Optimization restart 4/5, f = 732.4075668702678
Optimization restart 5/5, f = 732.4075668441158
[[13.   8.   5.  70.   3.   1.5]]

User function emissions called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 1.8259038951974815

Optimization restart 1/5, f = 709.6720981933329
Optimization restart 2/5, f = 734.4024021504349
Optimization restart 3/5, f = 734.402402135901
Optimization restart 4/5, f = 734.4024021365099
Optimization restart 5/5, f = 734.4024021359015
[[20.         25.          5.         58.10949147  1.          1.5       ]]

User function e

## Results

In [17]:
parameters_name = ['Size of the grid', 'Speed limit in the net (m/s)', 'Max vehicles speed (m/s)', 'Length of the roads (m)', 'Number of lanes', 'Accelleration (m/s^2)']

### Results on the random initialised emulator

In [18]:
results_random_init = bo_random_init.get_results()

In [19]:
results_random_init_df = pd.DataFrame(results_random_init.minimum_location.reshape(1,-1))
results_random_init_df.columns = parameters_name
results_random_init_df

,Size of the grid,Speed limit in the net (m/s),Max vehicles speed (m/s),Length of the roads (m),Number of lanes,Accelleration (m/s^2)
0,20.0,15.244704,5.0,70.0,1.0,1.5


In [20]:
results_random_init.minimum_value

1.19946831512822

### Results on the experimentally designed emulator with model variance

In [21]:
results_model_variance = bo_model_variance.get_results()

In [22]:
results_random_init_df = pd.DataFrame(results_model_variance.minimum_location.reshape(1,-1))
results_random_init_df.columns = parameters_name
results_random_init_df

,Size of the grid,Speed limit in the net (m/s),Max vehicles speed (m/s),Length of the roads (m),Number of lanes,Accelleration (m/s^2)
0,20.0,8.0,5.0,70.0,1.0,1.5


In [23]:
results_model_variance.minimum_value

1.19946831512822

### Results on the experimentally designed emulator with integrated variance reduction

In [24]:
results_integrated_variance = bo_integrated_variance.get_results()

In [25]:
results_random_init_df = pd.DataFrame(results_integrated_variance.minimum_location.reshape(1,-1))
results_random_init_df.columns = parameters_name
results_random_init_df

,Size of the grid,Speed limit in the net (m/s),Max vehicles speed (m/s),Length of the roads (m),Number of lanes,Accelleration (m/s^2)
0,20.0,25.0,5.0,70.0,1.0,1.5


In [26]:
results_integrated_variance.minimum_value

1.19946831512822

## Save stuff

In [27]:
with open(f'bayesian_optimization_new_outputs/bayesian_opt_results/results_random_init_emissions.pkl', "wb") as f:
     pickle.dump(results_random_init, f)

with open(f'bayesian_optimization_new_outputs/bayesian_opt_results/results_model_variance_emissions.pkl', "wb") as f:
     pickle.dump(results_model_variance, f)
        
with open(f'bayesian_optimization_new_outputs/bayesian_opt_results/results_integrated_variance_emissions.pkl', "wb") as f:
     pickle.dump(results_integrated_variance, f)

In [28]:
with open(f'bayesian_optimization_new_outputs/models/270_random_init_50_bay_opt_emissions.pkl', "wb") as f:
     pickle.dump(emukit_model_random_init, f)

with open(f'bayesian_optimization_new_outputs/models/270_model_variance_50_bay_emissions.pkl', "wb") as f:
     pickle.dump(emukit_model_variance, f)
        
with open(f'bayesian_optimization_new_outputs/models/270_integrated_variance_50_bay_emissions.pkl', "wb") as f:
     pickle.dump(emukit_model_integrated_variance, f)